## Задание

Данные имеют следующий вид:

    Clothing ID: id одежды
    Age: возраст клиента
    Title: заголовок отзыва по продукту
    Review Text: отзыв по продукту
    Rating: рейтинг продукта от 1 (худшее) до 5 (лучшее)
    Recommended IND: Бинарная переменная указывающая рекомендует ли клиент данный продукт (1) или нет (0)
    Positive Feedback Count: Количество клиентов, нашедших данный продукт хорошим
    Division Name, Department Name, Class Name - категории и подкатегории продукта
    
**_Rating_** - целевая переменная (y), которую требуется предсказывать по имеющимся характеристикам

train.csv - датасет для обучения
<br>
test.csv - датасет для теста, на котором будет производиться вычисление итоговой метрики, результаты предсказанного рейтинга на данном датасате необходимо прислать в csv формате (см. ниже)

Тестируемая метрика: **accuracy**

В качестве решения задания требуется прислать csv файл, а также ноутбук с решением задачи
<br>
Формат файла: Rating (один столбец). Пример см. ниже
<br>
Название файла: Surname_Name_#group.csv (Ivanov_Ivan_316.csv)

In [39]:
import pandas as pd
from pathlib import Path
from category_encoders import TargetEncoder
import nltk
import spacy
from nltk.corpus import stopwords
import re
import numpy as np
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [2]:
pd.DataFrame({'Rating': [5]*3})

,Rating
0,5
1,5
2,5


In [3]:
SEED = 42

In [4]:
df = pd.read_csv(Path('/Users/main/Мега_Задание_5/train.csv'))
train = df.copy()
train

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,867,47,Great top!,I got this top in an xs and the fit is great. ...,5,1,0,General,Tops,Knits
1,927,54,NaN,This poncho is very pretty and unique.it does ...,4,1,1,General Petite,Tops,Sweaters
2,1089,34,Gorgeous lace!,I was so pleased to receive this dress after p...,5,1,3,General,Dresses,Dresses
3,1072,56,Summer/fall wear,"I first spotted this on an retailer employee, ...",5,1,1,General Petite,Dresses,Dresses
4,869,66,New favorite summer blouse,The details on this blouse - lace inset in fro...,5,1,5,General,Tops,Knits
...,...,...,...,...,...,...,...,...,...,...
22306,877,65,Nostalgia tee,"Precious tee, washes well, runs large",4,1,1,General,Tops,Knits
22307,828,39,Great casual shirt,I love this shirt because it's casual yet fun!...,4,1,2,General,Tops,Blouses
22308,869,38,NaN,Beautiful top but it didn't fit. i am small ch...,3,1,0,General,Tops,Knits
22309,1096,48,Missed the mark!,Very disappointed: runs small in chest and too...,2,0,0,General,Dresses,Dresses


In [5]:
#переименуем столбцы для удобства
train = train.rename(columns = {'Clothing ID': 'id', 'Review Text': 'Review',
                        'Recommended IND': 'ind', 'Positive Feedback Count': 'PFC', 
                        'Division Name': 'Dev_Name', 'Department Name': 'Dep_Name', 'Class Name': 'Cl_Name'}, inplace = False)
train

,id,Age,Title,Review,Rating,ind,PFC,Dev_Name,Dep_Name,Cl_Name
0,867,47,Great top!,I got this top in an xs and the fit is great. ...,5,1,0,General,Tops,Knits
1,927,54,NaN,This poncho is very pretty and unique.it does ...,4,1,1,General Petite,Tops,Sweaters
2,1089,34,Gorgeous lace!,I was so pleased to receive this dress after p...,5,1,3,General,Dresses,Dresses
3,1072,56,Summer/fall wear,"I first spotted this on an retailer employee, ...",5,1,1,General Petite,Dresses,Dresses
4,869,66,New favorite summer blouse,The details on this blouse - lace inset in fro...,5,1,5,General,Tops,Knits
...,...,...,...,...,...,...,...,...,...,...
22306,877,65,Nostalgia tee,"Precious tee, washes well, runs large",4,1,1,General,Tops,Knits
22307,828,39,Great casual shirt,I love this shirt because it's casual yet fun!...,4,1,2,General,Tops,Blouses
22308,869,38,NaN,Beautiful top but it didn't fit. i am small ch...,3,1,0,General,Tops,Knits
22309,1096,48,Missed the mark!,Very disappointed: runs small in chest and too...,2,0,0,General,Dresses,Dresses


In [6]:
#отделяем target и основной датафрейм
y_train = train['Rating'] 
x_train = train.drop(['Rating'],axis = 1)
x_train

,id,Age,Title,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name
0,867,47,Great top!,I got this top in an xs and the fit is great. ...,1,0,General,Tops,Knits
1,927,54,NaN,This poncho is very pretty and unique.it does ...,1,1,General Petite,Tops,Sweaters
2,1089,34,Gorgeous lace!,I was so pleased to receive this dress after p...,1,3,General,Dresses,Dresses
3,1072,56,Summer/fall wear,"I first spotted this on an retailer employee, ...",1,1,General Petite,Dresses,Dresses
4,869,66,New favorite summer blouse,The details on this blouse - lace inset in fro...,1,5,General,Tops,Knits
...,...,...,...,...,...,...,...,...,...
22306,877,65,Nostalgia tee,"Precious tee, washes well, runs large",1,1,General,Tops,Knits
22307,828,39,Great casual shirt,I love this shirt because it's casual yet fun!...,1,2,General,Tops,Blouses
22308,869,38,NaN,Beautiful top but it didn't fit. i am small ch...,1,0,General,Tops,Knits
22309,1096,48,Missed the mark!,Very disappointed: runs small in chest and too...,0,0,General,Dresses,Dresses


### Заполнение NaN'ов

In [7]:
x_train.isnull().sum()

id             0
Age            0
Title       3642
Review       795
ind            0
PFC            0
Dev_Name      14
Dep_Name      14
Cl_Name       14
dtype: int64

In [8]:
mask = (x_train.Title.isnull()) & (x_train.Review.isnull())
x_train.loc[mask].ind.mean()

0.9130982367758187

In [9]:
#заполним NaN'ы в Title пустыми строчками
#логика за этим стоит такая: если человек не оставляет никаких комментариев, обычно он (или она) очень доволен продуктом и ставит ему оценку 5
#хотелось бы, чтобы модель уловила эту зависимость
mask = x_train.Title.isnull() == True
x_train.loc[mask, 'Title'] = ''
x_train.isnull().sum()

id            0
Age           0
Title         0
Review      795
ind           0
PFC           0
Dev_Name     14
Dep_Name     14
Cl_Name      14
dtype: int64

In [10]:
#заполним NaN'ы в Review пустыми строчками
#логика за этим стоит такая: если человек не оставляет никаких комментариев, обычно он (или она) очень доволен продуктом и ставит ему оценку 5
#хотелось бы, чтобы модель уловила эту зависимость
mask = x_train.Review.isnull() == True
x_train.loc[mask, 'Review'] = ''
x_train.isnull().sum()

id           0
Age          0
Title        0
Review       0
ind          0
PFC          0
Dev_Name    14
Dep_Name    14
Cl_Name     14
dtype: int64

In [11]:
#столбцов с Nan'ами в Dev_Name, Dep_Name, Cl_Name мало, поэтому можно заполнить их самыми популярными в этих столбцах значениями
list = ['Dev_Name', 'Dep_Name', 'Cl_Name']
for i in list:
    x_train[i].fillna(x_train[i].mode()[0], inplace = True)
x_train.isnull().sum()

id          0
Age         0
Title       0
Review      0
ind         0
PFC         0
Dev_Name    0
Dep_Name    0
Cl_Name     0
dtype: int64

### Преобразование категоральных фичей 

In [12]:
x_train

,id,Age,Title,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name
0,867,47,Great top!,I got this top in an xs and the fit is great. ...,1,0,General,Tops,Knits
1,927,54,,This poncho is very pretty and unique.it does ...,1,1,General Petite,Tops,Sweaters
2,1089,34,Gorgeous lace!,I was so pleased to receive this dress after p...,1,3,General,Dresses,Dresses
3,1072,56,Summer/fall wear,"I first spotted this on an retailer employee, ...",1,1,General Petite,Dresses,Dresses
4,869,66,New favorite summer blouse,The details on this blouse - lace inset in fro...,1,5,General,Tops,Knits
...,...,...,...,...,...,...,...,...,...
22306,877,65,Nostalgia tee,"Precious tee, washes well, runs large",1,1,General,Tops,Knits
22307,828,39,Great casual shirt,I love this shirt because it's casual yet fun!...,1,2,General,Tops,Blouses
22308,869,38,,Beautiful top but it didn't fit. i am small ch...,1,0,General,Tops,Knits
22309,1096,48,Missed the mark!,Very disappointed: runs small in chest and too...,0,0,General,Dresses,Dresses


In [13]:
encoder1 = TargetEncoder(smoothing=5)
encoder2 = TargetEncoder(smoothing=5)
encoder3 = TargetEncoder(smoothing=5)
x_train['Dev_Name'] = encoder1.fit_transform(x_train.Dev_Name, y_train)
x_train['Dep_Name'] = encoder2.fit_transform(x_train.Dep_Name, y_train)
x_train['Cl_Name'] = encoder3.fit_transform(x_train.Cl_Name, y_train)
x_train

,id,Age,Title,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name
0,867,47,Great top!,I got this top in an xs and the fit is great. ...,1,0,4.180976,4.176222,4.163545
1,927,54,,This poncho is very pretty and unique.it does ...,1,1,4.215999,4.176222,4.180133
2,1089,34,Gorgeous lace!,I was so pleased to receive this dress after p...,1,3,4.180976,4.156026,4.157991
3,1072,56,Summer/fall wear,"I first spotted this on an retailer employee, ...",1,1,4.215999,4.156026,4.157991
4,869,66,New favorite summer blouse,The details on this blouse - lace inset in fro...,1,5,4.180976,4.176222,4.163545
...,...,...,...,...,...,...,...,...,...
22306,877,65,Nostalgia tee,"Precious tee, washes well, runs large",1,1,4.180976,4.176222,4.163545
22307,828,39,Great casual shirt,I love this shirt because it's casual yet fun!...,1,2,4.180976,4.176222,4.160666
22308,869,38,,Beautiful top but it didn't fit. i am small ch...,1,0,4.180976,4.176222,4.163545
22309,1096,48,Missed the mark!,Very disappointed: runs small in chest and too...,0,0,4.180976,4.156026,4.157991


### Обработка текста

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/main/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
def preprocess_text(series_column, stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    ''' Simple text preprocessing
        
    Parameters:
        series_column - pandas.series
        stop_words - list of stop words
        allowed_postags - what types of words must be kept after lemmatization
        
    Return:
        preprocessed series
    '''
    series_column = series_column.str.lower()
    #series_column = series_column.replace(r'[^a-z]', ' ', regex = True)
    
    for i, text in series_column.iteritems():
        
        #drop emails
        text = re.sub('\S*@\S*\s?', '', text)
        # Remove distracting single quotes
        text = re.sub("\'", "", text)
        #keep only letters
        text = re.sub('[^a-z]', ' ', text)
        
        #drop whitespaces
        text = ' '.join(text.split())
        
        #normalization and drop stop words and too short ones
        doc = nlp(text)
        tokens = [token.lemma_ for token in doc if token.pos_ in allowed_postags
                                                   and token.lemma_ not in stop_words
                                                   and len(token.lemma_) > 2]
        
        #old approach but faster than spacy
        #tokens = mystem.lemmatize(text)
        #tokens = [token for token in tokens if token not in [' ', '\n'] 
        #                                      and token not in stop_words
        #                                      and len(token) > 2]
        
        text = ' '.join(tokens)
        series_column[i] = text
    
    return series_column

In [16]:
nlp = spacy.load('en', disable=['parser', 'ner'])
stop_words = stopwords.words('english')

In [17]:
%%time
x_train.Title = preprocess_text(x_train.Title, stop_words)
x_train.Review = preprocess_text(x_train.Review,stop_words)
x_train

CPU times: user 1min 33s, sys: 471 ms, total: 1min 33s
Wall time: 1min 34s


,id,Age,Title,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name
0,867,47,great top,get top fit great top little dark picture look...,1,0,4.180976,4.176222,4.163545
1,927,54,,poncho pretty unique run sized however style e...,1,1,4.215999,4.176222,4.180133
2,1089,34,gorgeous lace,pleased receive dress purchase online color vi...,1,3,4.180976,4.156026,4.157991
3,1072,56,summer fall wear,first spot retailer employee pair peasant top ...,1,1,4.215999,4.156026,4.157991
4,869,66,new favorite summer blouse,detail blouse lace front sheer look great pers...,1,5,4.180976,4.176222,4.163545
...,...,...,...,...,...,...,...,...,...
22306,877,65,,precious tee wash well run large,1,1,4.180976,4.176222,4.163545
22307,828,39,great casual shirt,love shirt casual first time wear receive seve...,1,2,4.180976,4.176222,4.160666
22308,869,38,,beautiful top fit small cheste wide shoulder n...,1,0,4.180976,4.176222,4.163545
22309,1096,48,miss mark,disappointed run small chest long fabric cheap...,0,0,4.180976,4.156026,4.157991


In [18]:
matrix_text = [sent.split() for sent in x_train.Title]

In [19]:
%%time
w2v = Word2Vec(matrix_text, size=100, sg=0, iter=10, workers=5, min_count = 0)

CPU times: user 854 ms, sys: 19.7 ms, total: 874 ms
Wall time: 701 ms


In [20]:
d = dict(zip(w2v.wv.vocab.keys(), [w2v.wv[word] for word in w2v.wv.vocab.keys()]))
data = pd.DataFrame.from_dict(d, orient='index')
data

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
great,-0.246592,0.024847,-0.050158,-0.202228,-0.249909,0.011104,0.035172,0.336391,0.283332,0.114944,...,0.187458,-0.258079,-0.096015,0.175569,0.201150,-0.168740,-0.019834,-0.091660,0.290175,-0.053896
top,-0.229319,0.036327,-0.065290,-0.178648,-0.262293,0.005997,0.039914,0.309059,0.294912,0.113454,...,0.177864,-0.252614,-0.094371,0.169423,0.194394,-0.184662,-0.036078,-0.120893,0.276942,-0.095415
gorgeous,-0.230736,0.033236,-0.061514,-0.173196,-0.260436,0.007125,0.027314,0.330475,0.310372,0.117759,...,0.176696,-0.258674,-0.098882,0.167386,0.222087,-0.189110,-0.022945,-0.093268,0.294120,-0.076261
lace,-0.180183,0.025926,-0.045458,-0.139904,-0.199766,0.006413,0.021397,0.245113,0.240109,0.094911,...,0.139327,-0.194615,-0.072227,0.134303,0.160598,-0.142705,-0.025854,-0.068760,0.218791,-0.064244
summer,-0.213125,0.025838,-0.052165,-0.186729,-0.225424,-0.008509,0.051355,0.291352,0.239172,0.089534,...,0.170630,-0.240916,-0.071304,0.139402,0.160901,-0.152379,-0.022230,-0.098967,0.234040,-0.073006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
modification,-0.015204,0.003697,-0.007222,-0.008559,-0.018383,0.003714,-0.002636,0.016990,0.017028,0.010751,...,0.012807,-0.018323,-0.005067,0.008552,0.017640,-0.011749,0.000789,-0.010378,0.023047,-0.008074
camel,-0.006313,-0.000949,0.003836,-0.005490,-0.004388,0.002761,0.004600,0.006450,0.000771,0.002012,...,0.007190,-0.003899,0.000691,0.006409,0.005445,-0.006019,0.003357,-0.004644,-0.000253,-0.000277
draped,-0.026221,0.000993,-0.004564,-0.018158,-0.027372,0.000252,0.007278,0.036046,0.034468,0.009703,...,0.021526,-0.027082,-0.009687,0.022125,0.025373,-0.026569,-0.005428,-0.011230,0.031697,-0.014241
horror,-0.004855,0.000136,0.003244,0.002532,0.001767,0.003282,0.000991,0.004929,-0.003909,-0.004752,...,-0.003968,0.002738,0.003540,-0.004253,-0.004439,-0.001013,-0.002735,-0.004076,0.004324,0.004197


In [21]:
'''
x_train_1 = x_train.copy()
x_train_1
'''

'\nx_train_1 = x_train.copy()\nx_train_1\n'

In [22]:
def line_2_vec(x):
    vec = np.zeros(100)
    #mat = np.zeros((1,100))
    for i in x.split():
        vec = vec + w2v.wv.get_vector(i)
        #mat = np.vstack((mat,vec))
    return vec

In [23]:
vec = x_train.Title.apply(line_2_vec)
vec_data = pd.DataFrame.from_records(vec.values,index=vec.index)
result = pd.concat([x_train,vec_data], axis = 1,sort = False)

In [24]:
x_train = result.drop(columns = ['Title'])

In [25]:
d = {}
for i in range(100):
    s = 'T_' + str(i)
    d[i] = s
x_train = x_train.rename(columns = d)
check = x_train
x_train

,id,Age,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name,T_0,T_1,...,T_90,T_91,T_92,T_93,T_94,T_95,T_96,T_97,T_98,T_99
0,867,47,get top fit great top little dark picture look...,1,0,4.180976,4.176222,4.163545,-0.475911,0.061174,...,0.365322,-0.510693,-0.190386,0.344992,0.395544,-0.353403,-0.055912,-0.212553,0.567117,-0.149311
1,927,54,poncho pretty unique run sized however style e...,1,1,4.215999,4.176222,4.180133,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,1089,34,pleased receive dress purchase online color vi...,1,3,4.180976,4.156026,4.157991,-0.410920,0.059162,...,0.316023,-0.453289,-0.171109,0.301690,0.382685,-0.331815,-0.048799,-0.162028,0.512911,-0.140505
3,1072,56,first spot retailer employee pair peasant top ...,1,1,4.215999,4.156026,4.157991,-0.719011,0.064793,...,0.577554,-0.785436,-0.276545,0.485696,0.563074,-0.511958,-0.088462,-0.312370,0.837297,-0.270265
4,869,66,detail blouse lace front sheer look great pers...,1,5,4.180976,4.176222,4.163545,-1.015079,0.079527,...,0.742019,-1.132138,-0.164108,0.681877,0.646498,-0.784404,-0.060667,-0.449629,0.967706,-0.302436
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,877,65,precious tee wash well run large,1,1,4.180976,4.176222,4.163545,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22307,828,39,love shirt casual first time wear receive seve...,1,2,4.180976,4.176222,4.160666,-0.694673,0.079559,...,0.541022,-0.740844,-0.271098,0.506021,0.564566,-0.510744,-0.067009,-0.292538,0.815394,-0.226336
22308,869,38,beautiful top fit small cheste wide shoulder n...,1,0,4.180976,4.176222,4.163545,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
22309,1096,48,disappointed run small chest long fabric cheap...,0,0,4.180976,4.156026,4.157991,-0.318493,0.031197,...,0.272340,-0.362270,-0.140572,0.238949,0.304991,-0.264995,-0.036143,-0.125153,0.399280,-0.117602


In [26]:
'''
x_train_1
title_text = [sent.split() for sent in x_train_1.Title]
w2v = Word2Vec(title_text, size=100, sg=0, iter=10, workers=5, min_count = 0)
d = dict(zip(w2v.wv.vocab.keys(), [w2v.wv[word] for word in w2v.wv.vocab.keys()]))
data = pd.DataFrame.from_dict(d, orient='index')
data
'''

"\nx_train_1\ntitle_text = [sent.split() for sent in x_train_1.Title]\nw2v = Word2Vec(title_text, size=100, sg=0, iter=10, workers=5, min_count = 0)\nd = dict(zip(w2v.wv.vocab.keys(), [w2v.wv[word] for word in w2v.wv.vocab.keys()]))\ndata = pd.DataFrame.from_dict(d, orient='index')\ndata\n"

In [27]:
'''
def line_2_vec(x):
    vec = np.zeros(100)
    #mat = np.zeros((1,100))
    for i in x.split():
        vec = vec + w2v.wv.get_vector(i)
        #mat = np.vstack((mat,vec))
    return vec
vec = x_train_1.Title.apply(line_2_vec)
vec_data = pd.DataFrame.from_records(vec.values,index=vec.index)
result = pd.concat([x_train_1,vec_data], axis = 1,sort = False)
x_train_1 = result.drop(columns = ['Title'])
'''

"\ndef line_2_vec(x):\n    vec = np.zeros(100)\n    #mat = np.zeros((1,100))\n    for i in x.split():\n        vec = vec + w2v.wv.get_vector(i)\n        #mat = np.vstack((mat,vec))\n    return vec\nvec = x_train_1.Title.apply(line_2_vec)\nvec_data = pd.DataFrame.from_records(vec.values,index=vec.index)\nresult = pd.concat([x_train_1,vec_data], axis = 1,sort = False)\nx_train_1 = result.drop(columns = ['Title'])\n"

In [28]:
'''
d = {}
for i in range(100):
    s = 'T_' + str(i)
    d[i] = s
x_train_1 = x_train_1.rename(columns = d)
x_train_1
'''

"\nd = {}\nfor i in range(100):\n    s = 'T_' + str(i)\n    d[i] = s\nx_train_1 = x_train_1.rename(columns = d)\nx_train_1\n"

In [29]:
'''
review_text = [sent.split() for sent in x_train_1.Review]
w2v = Word2Vec(review_text, size=100, sg=0, iter=10, workers=5, min_count = 0)
d = dict(zip(w2v.wv.vocab.keys(), [w2v.wv[word] for word in w2v.wv.vocab.keys()]))
data = pd.DataFrame.from_dict(d, orient='index')
data
'''

"\nreview_text = [sent.split() for sent in x_train_1.Review]\nw2v = Word2Vec(review_text, size=100, sg=0, iter=10, workers=5, min_count = 0)\nd = dict(zip(w2v.wv.vocab.keys(), [w2v.wv[word] for word in w2v.wv.vocab.keys()]))\ndata = pd.DataFrame.from_dict(d, orient='index')\ndata\n"

In [30]:
'''
def line_2_vec(x):
    vec = np.zeros(100)
    #mat = np.zeros((1,100))
    for i in x.split():
        vec = vec + w2v.wv.get_vector(i)
        #mat = np.vstack((mat,vec))
    return vec
vec = x_train_1.Review.apply(line_2_vec)
vec_data = pd.DataFrame.from_records(vec.values,index=vec.index)
result = pd.concat([x_train_1,vec_data], axis = 1,sort = False)
x_train_1 = result.drop(columns = ['Review'])
x_train_1
'''

"\ndef line_2_vec(x):\n    vec = np.zeros(100)\n    #mat = np.zeros((1,100))\n    for i in x.split():\n        vec = vec + w2v.wv.get_vector(i)\n        #mat = np.vstack((mat,vec))\n    return vec\nvec = x_train_1.Review.apply(line_2_vec)\nvec_data = pd.DataFrame.from_records(vec.values,index=vec.index)\nresult = pd.concat([x_train_1,vec_data], axis = 1,sort = False)\nx_train_1 = result.drop(columns = ['Review'])\nx_train_1\n"

In [31]:
'''
d = {}
for i in range(100):
    s = 'R1_' + str(i)
    d[i] = s
x_train_1 = x_train_1.rename(columns = d)
x_train_1
'''

"\nd = {}\nfor i in range(100):\n    s = 'R1_' + str(i)\n    d[i] = s\nx_train_1 = x_train_1.rename(columns = d)\nx_train_1\n"

In [32]:
#X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train_1, y_train, test_size = 0.2, random_state = SEED)

In [33]:
'''
%%time
LR = LogisticRegression(solver = 'newton-cg',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
'''

"\n%%time\nLR = LogisticRegression(solver = 'newton-cg',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)\nLR.fit(X_train_,y_train_)\ntrain_pred = LR.predict(X_train_)\ntest_pred = LR.predict(X_test_)\nprint('Train_accuracy =',accuracy_score(y_train_,train_pred))\nprint('Test_accuracy =',accuracy_score(y_test_,test_pred))\n"

In [34]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=0.05, ngram_range=(1, 1))
matrix = vectorizer.fit_transform(x_train.Review)
print(matrix.shape)
matrix.toarray()[0:1,:]

(22311, 104)


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.2142931 , 0.        , 0.        , 0.32114986, 0.        ,
        0.        , 0.53138714, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.3147062 ,
        0.        , 0.23182717, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.39282407, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

In [35]:
review_vec = pd.DataFrame(matrix.toarray())
result = pd.concat([x_train,review_vec], axis = 1,sort = False)
result

,id,Age,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name,T_0,T_1,...,94,95,96,97,98,99,100,101,102,103
0,867,47,get top fit great top little dark picture look...,1,0,4.180976,4.176222,4.163545,-0.475911,0.061174,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0
1,927,54,poncho pretty unique run sized however style e...,1,1,4.215999,4.176222,4.180133,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.111130,0.151031,0.00000,0.0
2,1089,34,pleased receive dress purchase online color vi...,1,3,4.180976,4.156026,4.157991,-0.410920,0.059162,...,0.242481,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0
3,1072,56,first spot retailer employee pair peasant top ...,1,1,4.215999,4.156026,4.157991,-0.719011,0.064793,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.166000,0.000000,0.00000,0.0
4,869,66,detail blouse lace front sheer look great pers...,1,5,4.180976,4.176222,4.163545,-1.015079,0.079527,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.130073,0.353549,0.20018,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,877,65,precious tee wash well run large,1,1,4.180976,4.176222,4.163545,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.544164,0.00000,0.0
22307,828,39,love shirt casual first time wear receive seve...,1,2,4.180976,4.176222,4.160666,-0.694673,0.079559,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.352959,0.000000,0.00000,0.0
22308,869,38,beautiful top fit small cheste wide shoulder n...,1,0,4.180976,4.176222,4.163545,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.281991,0.000000,0.00000,0.0
22309,1096,48,disappointed run small chest long fabric cheap...,0,0,4.180976,4.156026,4.157991,-0.318493,0.031197,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.237357,0.00000,0.0


In [36]:
x_train = result.drop(columns = ['Review'])
x_train

,id,Age,ind,PFC,Dev_Name,Dep_Name,Cl_Name,T_0,T_1,T_2,...,94,95,96,97,98,99,100,101,102,103
0,867,47,1,0,4.180976,4.176222,4.163545,-0.475911,0.061174,-0.115448,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0
1,927,54,1,1,4.215999,4.176222,4.180133,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.111130,0.151031,0.00000,0.0
2,1089,34,1,3,4.180976,4.156026,4.157991,-0.410920,0.059162,-0.106971,...,0.242481,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0
3,1072,56,1,1,4.215999,4.156026,4.157991,-0.719011,0.064793,-0.172736,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.166000,0.000000,0.00000,0.0
4,869,66,1,5,4.180976,4.176222,4.163545,-1.015079,0.079527,-0.230603,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.130073,0.353549,0.20018,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,877,65,1,1,4.180976,4.176222,4.163545,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.544164,0.00000,0.0
22307,828,39,1,2,4.180976,4.176222,4.160666,-0.694673,0.079559,-0.161842,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.352959,0.000000,0.00000,0.0
22308,869,38,1,0,4.180976,4.176222,4.163545,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.281991,0.000000,0.00000,0.0
22309,1096,48,0,0,4.180976,4.156026,4.157991,-0.318493,0.031197,-0.075887,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.237357,0.00000,0.0


In [37]:
d = {}
for i in range(104):
    s = 'R_' + str(i)
    d[i] = s
x_train = x_train.rename(columns = d)
x_train

,id,Age,ind,PFC,Dev_Name,Dep_Name,Cl_Name,T_0,T_1,T_2,...,R_94,R_95,R_96,R_97,R_98,R_99,R_100,R_101,R_102,R_103
0,867,47,1,0,4.180976,4.176222,4.163545,-0.475911,0.061174,-0.115448,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0
1,927,54,1,1,4.215999,4.176222,4.180133,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.111130,0.151031,0.00000,0.0
2,1089,34,1,3,4.180976,4.156026,4.157991,-0.410920,0.059162,-0.106971,...,0.242481,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.00000,0.0
3,1072,56,1,1,4.215999,4.156026,4.157991,-0.719011,0.064793,-0.172736,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.166000,0.000000,0.00000,0.0
4,869,66,1,5,4.180976,4.176222,4.163545,-1.015079,0.079527,-0.230603,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.130073,0.353549,0.20018,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22306,877,65,1,1,4.180976,4.176222,4.163545,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.544164,0.00000,0.0
22307,828,39,1,2,4.180976,4.176222,4.160666,-0.694673,0.079559,-0.161842,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.352959,0.000000,0.00000,0.0
22308,869,38,1,0,4.180976,4.176222,4.163545,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.281991,0.000000,0.00000,0.0
22309,1096,48,0,0,4.180976,4.156026,4.157991,-0.318493,0.031197,-0.075887,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.237357,0.00000,0.0


In [38]:
dic = {}

In [ ]:
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
GBC = GradientBoostingClassifier(n_estimators = 1000, random_state=42)
GBC.fit(X_train_,y_train_)
train_pred = GBC.predict(X_train_)
test_pred = GBC.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))

In [40]:
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
ABC = AdaBoostClassifier(random_state =SEED)
ABC.fit(X_train_,y_train_)
train_pred = ABC.predict(X_train_)
test_pred = ABC.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['ABC'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))

Train_accuracy = 0.5858359480053787
Test_accuracy = 0.5888415863768766
CPU times: user 7.41 s, sys: 136 ms, total: 7.55 s
Wall time: 7.56 s


In [41]:
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
RFC = RandomForestClassifier(max_depth = 19,random_state =SEED)
RFC.fit(X_train_,y_train_)
train_pred = RFC.predict(X_train_)
test_pred = RFC.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['RFC'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))

Train_accuracy = 0.9196548632900045
Test_accuracy = 0.6435133318395698
CPU times: user 9.26 s, sys: 28.5 ms, total: 9.28 s
Wall time: 9.29 s


In [42]:
dic

{'ABC': (0.5858359480053787, 0.5888415863768766),
 'RFC': (0.9196548632900045, 0.6435133318395698)}

In [43]:
'''
maximum, ind = 0, 0
for i in range(14,25):
    X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
    RFC = RandomForestClassifier(max_depth = i,random_state =SEED)
    RFC.fit(X_train_,y_train_)
    train_pred = RFC.predict(X_train_)
    test_pred = RFC.predict(X_test_)
    if accuracy_score(y_test_,test_pred) > maximum:
        maximum = accuracy_score(y_test_,test_pred)
        ind = i
print('Best:[',ind,']Test_accuracy =',maximum)
    #print('[',i,']Train_accuracy =',accuracy_score(y_train_,train_pred))
    #print('[',i,']Test_accuracy =',accuracy_score(y_test_,test_pred))
'''

"\nmaximum, ind = 0, 0\nfor i in range(14,25):\n    X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)\n    RFC = RandomForestClassifier(max_depth = i,random_state =SEED)\n    RFC.fit(X_train_,y_train_)\n    train_pred = RFC.predict(X_train_)\n    test_pred = RFC.predict(X_test_)\n    if accuracy_score(y_test_,test_pred) > maximum:\n        maximum = accuracy_score(y_test_,test_pred)\n        ind = i\nprint('Best:[',ind,']Test_accuracy =',maximum)\n    #print('[',i,']Train_accuracy =',accuracy_score(y_train_,train_pred))\n    #print('[',i,']Test_accuracy =',accuracy_score(y_test_,test_pred))\n"

In [44]:
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
LR = LogisticRegression(solver = 'liblinear',penalty='l1',random_state = SEED,max_iter=10000)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['LR_liblinear'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))

Train_accuracy = 0.6667413715822501
Test_accuracy = 0.6607663006946001
CPU times: user 13.4 s, sys: 51.1 ms, total: 13.4 s
Wall time: 13.3 s


In [45]:
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
LR = LogisticRegression(solver = 'newton-cg',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['LR_newton-cg'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))

Train_accuracy = 0.6724562976243836
Test_accuracy = 0.6587497199193367
CPU times: user 914 ms, sys: 258 ms, total: 1.17 s
Wall time: 2min 19s


In [46]:
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
LR = LogisticRegression(solver = 'sag',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['LR_sag'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))

Train_accuracy = 0.6407440609592111
Test_accuracy = 0.6439614609007394
CPU times: user 3min 24s, sys: 378 ms, total: 3min 24s
Wall time: 3min 24s


In [47]:
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
LR = LogisticRegression(solver = 'lbfgs',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['LR_lbfgs'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))

Train_accuracy = 0.6652846257283729
Test_accuracy = 0.658525655388752
CPU times: user 951 ms, sys: 223 ms, total: 1.17 s
Wall time: 3min 25s


In [48]:
'''
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train.drop(columns = ['Dev_Name','Dep_Name','Cl_Name']),
                                                        y_train, test_size = 0.2, random_state = 42)
LR = LogisticRegression(solver = 'lbfgs',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['LR_lbfgs_drop3'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))
'''

"\n%%time\nX_train_, X_test_, y_train_, y_test_ = train_test_split(x_train.drop(columns = ['Dev_Name','Dep_Name','Cl_Name']),\n                                                        y_train, test_size = 0.2, random_state = 42)\nLR = LogisticRegression(solver = 'lbfgs',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)\nLR.fit(X_train_,y_train_)\ntrain_pred = LR.predict(X_train_)\ntest_pred = LR.predict(X_test_)\nprint('Train_accuracy =',accuracy_score(y_train_,train_pred))\nprint('Test_accuracy =',accuracy_score(y_test_,test_pred))\ndic['LR_lbfgs_drop3'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))\n"

In [49]:
'''
%%time
X_train_, X_test_, y_train_, y_test_ = train_test_split(check.drop(columns = ['Review']),
                                                        y_train, test_size = 0.2, random_state = 42)
LR = LogisticRegression(solver = 'lbfgs',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
dic['LR_lbfgs_drop_Review'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))
'''

"\n%%time\nX_train_, X_test_, y_train_, y_test_ = train_test_split(check.drop(columns = ['Review']),\n                                                        y_train, test_size = 0.2, random_state = 42)\nLR = LogisticRegression(solver = 'lbfgs',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)\nLR.fit(X_train_,y_train_)\ntrain_pred = LR.predict(X_train_)\ntest_pred = LR.predict(X_test_)\nprint('Train_accuracy =',accuracy_score(y_train_,train_pred))\nprint('Test_accuracy =',accuracy_score(y_test_,test_pred))\ndic['LR_lbfgs_drop_Review'] = (accuracy_score(y_train_,train_pred),accuracy_score(y_test_,test_pred))\n"

In [50]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
only_5_train = y_train_.apply(lambda x: 5)
only_5_test = y_test_.apply(lambda x: 5)
dic['Baseline'] = (accuracy_score(y_train_, only_5_train),accuracy_score(y_test_, only_5_test)) #получается наша модель чего-то стоит)

In [51]:
dic

{'ABC': (0.5858359480053787, 0.5888415863768766),
 'RFC': (0.9196548632900045, 0.6435133318395698),
 'LR_liblinear': (0.6667413715822501, 0.6607663006946001),
 'LR_newton-cg': (0.6724562976243836, 0.6587497199193367),
 'LR_sag': (0.6407440609592111, 0.6439614609007394),
 'LR_lbfgs': (0.6652846257283729, 0.658525655388752),
 'Baseline': (0.5602868668758404, 0.5610575845843603)}

### Преобразование test.csv

In [52]:
df = pd.read_csv(Path('/Users/main/Мега_Задание_5/test.csv'))
test = df.copy()
test

,Clothing ID,Age,Title,Review Text,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,1078,31,"Beautiful, but does not fit as pictured",Saw this dress in the blue motif and thought i...,1,0,General,Dresses,Dresses
1,830,28,NaN,I can not imagine the cut in the waist being f...,0,4,General,Tops,Blouses
2,508,43,NaN,NaN,1,0,Initmates,Intimate,Sleep
3,834,54,So chic!,This is a wow! so chic...reminds of something ...,1,2,General Petite,Tops,Blouses
4,651,33,NaN,"This is a very nice bra, upset though because ...",1,0,Initmates,Intimate,Intimates
...,...,...,...,...,...,...,...,...,...
1170,866,26,Quality and style strange,"I liked this top, i really did. i also liked t...",0,2,General,Tops,Knits
1171,868,44,Versatile top,"Bought in the store, will look great with blac...",1,0,General,Tops,Knits
1172,858,31,Hole in it after a month,I have had this shirt for a little over a mont...,0,0,General Petite,Tops,Knits
1173,1082,40,Great for summer to fall,The issues the other reviewers mentioned didn'...,1,0,General,Dresses,Dresses


In [53]:
test = test.rename(columns = {'Clothing ID': 'id', 'Review Text': 'Review',
                        'Recommended IND': 'ind', 'Positive Feedback Count': 'PFC', 
                        'Division Name': 'Dev_Name', 'Department Name': 'Dep_Name', 'Class Name': 'Cl_Name'}, inplace = False)
test

,id,Age,Title,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name
0,1078,31,"Beautiful, but does not fit as pictured",Saw this dress in the blue motif and thought i...,1,0,General,Dresses,Dresses
1,830,28,NaN,I can not imagine the cut in the waist being f...,0,4,General,Tops,Blouses
2,508,43,NaN,NaN,1,0,Initmates,Intimate,Sleep
3,834,54,So chic!,This is a wow! so chic...reminds of something ...,1,2,General Petite,Tops,Blouses
4,651,33,NaN,"This is a very nice bra, upset though because ...",1,0,Initmates,Intimate,Intimates
...,...,...,...,...,...,...,...,...,...
1170,866,26,Quality and style strange,"I liked this top, i really did. i also liked t...",0,2,General,Tops,Knits
1171,868,44,Versatile top,"Bought in the store, will look great with blac...",1,0,General,Tops,Knits
1172,858,31,Hole in it after a month,I have had this shirt for a little over a mont...,0,0,General Petite,Tops,Knits
1173,1082,40,Great for summer to fall,The issues the other reviewers mentioned didn'...,1,0,General,Dresses,Dresses


In [54]:
x_test = test
mask = x_test.Title.isnull() == True
x_test.loc[mask, 'Title'] = ''
#x_test.isnull().sum()
mask = x_test.Review.isnull() == True
x_test.loc[mask, 'Review'] = ''
#x_test.isnull().sum()
list = ['Dev_Name', 'Dep_Name', 'Cl_Name']
for i in list:
    x_test[i].fillna(x_test[i].mode()[0], inplace = True)
#x_test.isnull().sum()

#что не так с Dev_Name??? количество различных одинаково
x_test['Dev_Name'] = encoder1.transform(x_test.Dev_Name)
x_test['Dep_Name'] = encoder2.transform(x_test.Dep_Name)
x_test['Cl_Name'] = encoder3.transform(x_test.Cl_Name)
#не понимаю почему, но encoder1 никак не хотел работать
x_test['Dev_Name'] = x_test.Dev_Name.apply(lambda x: 4.285614 if x == 'Initmates' else 4.180976 if x == 'General' else 4.215999)
#x_test


In [55]:
%%time
x_test.Title = preprocess_text(x_test.Title, stop_words)
x_test.Review = preprocess_text(x_test.Review,stop_words)
x_test

CPU times: user 3.81 s, sys: 12 ms, total: 3.83 s
Wall time: 3.83 s


,id,Age,Title,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name
0,1078,31,beautiful fit picture,see dress blue motif think would perfect summe...,1,0,4.215999,4.156026,4.157991
1,830,28,,imagine cut waist flattering order still would...,0,4,4.215999,4.176222,4.160666
2,508,43,,,1,0,4.215999,4.282596,4.267606
3,834,54,chic,wow chic remind would buy worth penny even hus...,1,2,4.215999,4.176222,4.160666
4,651,33,,nice bra upset though second time order retail...,1,0,4.215999,4.282596,4.265734
...,...,...,...,...,...,...,...,...,...
1170,866,26,quality style strange,like top really also like subtle shimmer fabri...,0,2,4.215999,4.176222,4.163545
1171,868,44,versatile top,buy store look great black dark blue jean fun ...,1,0,4.215999,4.176222,4.163545
1172,858,31,hole month,shirt little month wear maybe time small hole ...,0,0,4.215999,4.176222,4.163545
1173,1082,40,great summer fall,issue reviewer mention bother prior year versi...,1,0,4.215999,4.156026,4.157991


In [56]:
matrix_text_ = [sent.split() for sent in x_test.Title]
w2v = Word2Vec(matrix_text_, size=100, sg=0, iter=10, workers=5, min_count = 0)
vec = x_test.Title.apply(line_2_vec)
vec_data = pd.DataFrame.from_records(vec.values,index=vec.index)
result = pd.concat([x_test,vec_data], axis = 1,sort = False)
result

,id,Age,Title,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name,0,...,90,91,92,93,94,95,96,97,98,99
0,1078,31,beautiful fit picture,see dress blue motif think would perfect summe...,1,0,4.215999,4.156026,4.157991,0.007545,...,0.007030,-0.002772,-0.005604,0.002867,0.001107,-0.006470,0.000607,-0.001542,-0.001261,-0.000053
1,830,28,,imagine cut waist flattering order still would...,0,4,4.215999,4.176222,4.160666,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,508,43,,,1,0,4.215999,4.282596,4.267606,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,834,54,chic,wow chic remind would buy worth penny even hus...,1,2,4.215999,4.176222,4.160666,-0.004632,...,0.002296,0.004963,-0.004252,0.004578,0.001083,0.002920,-0.001928,0.001648,0.000844,-0.000473
4,651,33,,nice bra upset though second time order retail...,1,0,4.215999,4.282596,4.265734,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,866,26,quality style strange,like top really also like subtle shimmer fabri...,0,2,4.215999,4.176222,4.163545,-0.003988,...,-0.002292,-0.004231,0.008969,0.002466,0.005089,0.000114,-0.007087,0.005459,-0.002262,0.001414
1171,868,44,versatile top,buy store look great black dark blue jean fun ...,1,0,4.215999,4.176222,4.163545,-0.005776,...,0.003432,0.000945,-0.001597,0.001166,0.001605,0.003588,0.004462,-0.000463,-0.003286,0.006467
1172,858,31,hole month,shirt little month wear maybe time small hole ...,0,0,4.215999,4.176222,4.163545,-0.000052,...,-0.002265,0.003497,0.002623,0.007734,-0.002553,-0.004046,0.006252,-0.003753,0.005900,0.006391
1173,1082,40,great summer fall,issue reviewer mention bother prior year versi...,1,0,4.215999,4.156026,4.157991,0.005223,...,0.001200,0.010117,0.002922,0.001892,-0.000748,-0.001897,0.002123,0.004985,0.010342,0.007628


In [57]:
x_test = result.drop(columns = ['Title'])
#x_test
d = {}
for i in range(100):
    s = 'T_' + str(i)
    d[i] = s
x_test = x_test.rename(columns = d)
#x_test


In [58]:
vectorizer = TfidfVectorizer(max_df=0.95, min_df=0.05, ngram_range=(1, 1))
matrix = vectorizer.fit_transform(x_test.Review)
review_vec = pd.DataFrame(matrix.toarray())
result = pd.concat([x_test,review_vec], axis = 1,sort = False)
result

,id,Age,Review,ind,PFC,Dev_Name,Dep_Name,Cl_Name,T_0,T_1,...,93,94,95,96,97,98,99,100,101,102
0,1078,31,see dress blue motif think would perfect summe...,1,0,4.215999,4.156026,4.157991,0.007545,-0.002590,...,0.206782,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.153879
1,830,28,imagine cut waist flattering order still would...,0,4,4.215999,4.176222,4.160666,0.000000,0.000000,...,0.000000,0.000000,0.0,0.407934,0.0,0.0,0.000000,0.000000,0.000000,0.313172
2,508,43,,1,0,4.215999,4.282596,4.267606,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,834,54,wow chic remind would buy worth penny even hus...,1,2,4.215999,4.176222,4.160666,-0.004632,-0.001392,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.413936,0.349741
4,651,33,nice bra upset though second time order retail...,1,0,4.215999,4.282596,4.265734,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,866,26,like top really also like subtle shimmer fabri...,0,2,4.215999,4.176222,4.163545,-0.003988,-0.003528,...,0.000000,0.148749,0.0,0.000000,0.0,0.0,0.000000,0.141185,0.000000,0.000000
1171,868,44,buy store look great black dark blue jean fun ...,1,0,4.215999,4.176222,4.163545,-0.005776,0.000565,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.133843,0.000000,0.000000,0.000000
1172,858,31,shirt little month wear maybe time small hole ...,0,0,4.215999,4.176222,4.163545,-0.000052,-0.001739,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.139427,0.373139,0.000000,0.000000
1173,1082,40,issue reviewer mention bother prior year versi...,1,0,4.215999,4.156026,4.157991,0.005223,0.011535,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.422205,0.178364


In [59]:
x_test = result.drop(columns = ['Review'])
#x_test
d = {}
for i in range(103):
    s = 'R_' + str(i)
    d[i] = s
x_test = x_test.rename(columns = d)
x_test

,id,Age,ind,PFC,Dev_Name,Dep_Name,Cl_Name,T_0,T_1,T_2,...,R_93,R_94,R_95,R_96,R_97,R_98,R_99,R_100,R_101,R_102
0,1078,31,1,0,4.215999,4.156026,4.157991,0.007545,-0.002590,-0.007263,...,0.206782,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.153879
1,830,28,0,4,4.215999,4.176222,4.160666,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.407934,0.0,0.0,0.000000,0.000000,0.000000,0.313172
2,508,43,1,0,4.215999,4.282596,4.267606,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
3,834,54,1,2,4.215999,4.176222,4.160666,-0.004632,-0.001392,-0.002833,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.413936,0.349741
4,651,33,1,0,4.215999,4.282596,4.265734,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1170,866,26,0,2,4.215999,4.176222,4.163545,-0.003988,-0.003528,0.002886,...,0.000000,0.148749,0.0,0.000000,0.0,0.0,0.000000,0.141185,0.000000,0.000000
1171,868,44,1,0,4.215999,4.176222,4.163545,-0.005776,0.000565,-0.007725,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.133843,0.000000,0.000000,0.000000
1172,858,31,0,0,4.215999,4.176222,4.163545,-0.000052,-0.001739,0.002169,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.139427,0.373139,0.000000,0.000000
1173,1082,40,1,0,4.215999,4.156026,4.157991,0.005223,0.011535,0.001516,...,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.422205,0.178364


In [60]:
x_test['R_103'] = 0

In [80]:
#x_test
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
LR = LogisticRegression(solver = 'newton-cg',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))
#ans = pd.DataFrame(LR.predict(x_test))
#ans = ans.rename( columns = {0:'Rating'})
#ans

Train_accuracy = 0.6724562976243836
Test_accuracy = 0.6587497199193367


In [81]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.2, random_state = 42)
LR_lib = LogisticRegression(solver = 'liblinear',penalty='l1',random_state = SEED,max_iter=10000)
LR_lib.fit(X_train_,y_train_)
train_pred = LR_lib.predict(X_train_)
test_pred = LR_lib.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))

Train_accuracy = 0.6667413715822501
Test_accuracy = 0.6607663006946001


In [82]:
y_LR_newton = LR.predict(x_test)
print(y_LR_newton)
y_LR_liblinear = LR_lib.predict(x_test)
print(y_LR_liblinear)
y_check = np.abs(y_LR_newton - y_LR_liblinear)
pd.Series(y_check).value_counts()

[5 2 5 ... 3 5 5]
[5 2 5 ... 2 5 5]


0    1111
1      54
2      10
dtype: int64

In [83]:
ans = pd.DataFrame(y_LR_liblinear)
ans = ans.rename(columns = {0:'Rating'})
ans

,Rating
0,5
1,2
2,5
3,5
4,5
...,...
1170,3
1171,5
1172,2
1173,5


In [86]:
ans.to_csv('Bayborodin_Igor_523.csv',index = False)

In [87]:
df_ = pd.read_csv(Path('/Users/main/Мега_Задание_5/Bayborodin_Igor_523.csv'))
df_

,Rating
0,5
1,2
2,5
3,5
4,5
...,...
1170,3
1171,5
1172,2
1173,5


Итог работы очень странный: вроде как liblinear подходит лучше для маленьких датафреймов, да и работает по принципу one-versus-rest, что не очень подходит для нашей задачи. Но он показывает accuracy на test лучше, чем все остальные (AdaBoostClassifier, RandomForestClassifier, все остальные "решатели" LogisticRegression). Поэтому ничего не остается, кроме как выбрать его решение для test.csv. Да на train-test-split 80%-20% разница в accuracy на test между liblinear и newton-cg < 0,002, но с другой стороны первый всё же лучше. В общем, я оставил ответ от liblinear.

In [78]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.1, random_state = 42)
LR = LogisticRegression(solver = 'newton-cg',penalty='l2',random_state = SEED,max_iter=10000,n_jobs = -1)
LR.fit(X_train_,y_train_)
train_pred = LR.predict(X_train_)
test_pred = LR.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))

Train_accuracy = 0.6729418795756761
Test_accuracy = 0.6532258064516129


In [79]:
X_train_, X_test_, y_train_, y_test_ = train_test_split(x_train, y_train, test_size = 0.1, random_state = 42)
LR_lib = LogisticRegression(solver = 'liblinear',penalty='l1',random_state = SEED,max_iter=10000)
LR_lib.fit(X_train_,y_train_)
train_pred = LR_lib.predict(X_train_)
test_pred = LR_lib.predict(X_test_)
print('Train_accuracy =',accuracy_score(y_train_,train_pred))
print('Test_accuracy =',accuracy_score(y_test_,test_pred))

Train_accuracy = 0.667563125653668
Test_accuracy = 0.6505376344086021
